In [1]:
import time
from utils import randomize_wait
from whatsapp import WhatsappDriver

start_message = "Yinlin AI is now online!"
print(start_message)

driver = WhatsappDriver()
driver.start_webdriver_and_login()


Yinlin AI is now online!
Please scan the QR code to log in to WhatsApp Web.
Logged in successfully


In [7]:
import time
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException
from utils import randomize_wait

unread_contacts = []
unread_elements = driver.driver.find_elements(
    By.CSS_SELECTOR, 'div._ahlk span[aria-label*="unread message"]'
)
for element in unread_elements:
    # Navigate to the contact's parent element to 
    # get the contact name or other info
    contact_element = element.find_element(
        By.XPATH, './../../../../../div/div'
    )
    contact_name = contact_element.find_element(
        By.CSS_SELECTOR, 'span[title]'
    ).text
    unread_contacts.append(contact_name)
print(contact_name)

JJBA
